In [1]:
import traceback

import nltk
from nltk.corpus import stopwords
import re

import pandas as pd

from helpers.zefix_scraper import scrape_company

#nltk.download()        # run once

def stringify_list(l):
    return ' '.join(l)

def tokenize_interest_pipeline(interest):
    def remove_parenthesis(text):
        return re.sub("\(.*\)?",'', text).strip()

    def tokenize(interest):
        return nltk.word_tokenize(interest)
    
    def remove_stopwords(sentence):
        stop_words = stopwords.words(['german', 'french'])
        return [w for w in sentence if w.lower() not in stop_words]
    
    ans = remove_parenthesis(interest)
    ans = tokenize(ans)
    ans = remove_stopwords(ans)
    
    return ans

def more_one_letter_pipeline(interest):
    ans = tokenize_interest_pipeline(interest)
    
    return [token for token in ans if len(token) > 1]

dark_words = ['AG','SA']

def remove_dark_words_pipeline(interest):
    return [token for token in more_one_letter_pipeline(interest) if token.upper() not in dark_words]

def string_pipeline(s):
    return stringify_list(remove_dark_words_pipeline(s))

In [2]:
interests = pd.read_json('data/all_interests.json')
interests['interest_name_clean'] = interests.interest_name.apply(string_pipeline)
interests = interests.sort_values(by="index")
interests.head(5)

,data_source,first_name,index,interest_name,last_name,position,interest_name_clean
1980,geneva_parliament,Gabriel,1,Commission administrative du Foyer Eynard-Fati...,Barrillier,Membre,Commission administrative Foyer Eynard-Fatio E...
1981,geneva_parliament,Gabriel,1,Conseil de fondation de la SGIPA ...,Barrillier,Membre,Conseil fondation SGIPA
1982,geneva_parliament,Gabriel,1,Conseil de la Fondation Pro Juventute,Barrillier,,Conseil Fondation Pro Juventute
1989,geneva_parliament,David,2,Association Nicolas Bogueret ...,Amsler,Membre du comité,Association Nicolas Bogueret
1983,geneva_parliament,David,2,Amsler Bombeli et associés SA ...,Amsler,Administrateur,Amsler Bombeli associés


In [ ]:
scraped_companies = []
failed = []
for _, row in interests.iterrows():
    try:
        data = scrape_company(row.interest_name_clean) 
        if data is None:
            continue
        data["source"] = {
            "first_name": row.first_name,
            "last_name": row.last_name,
            "interest_name": row.interest_name,
            "data_source": row.data_source,
            "index": row["index"]
        }
        print(row["index"])    
        scraped_companies.append(data)
    except Exception as e:
        print("Error: " + str(e))
        traceback.print_exc()
        failed.append(row["index"])


2


In [74]:
open("scraped_companies_2.json", "w").write(json.dumps(scraped_companies2))

130348

In [76]:
len(scraped_companies2)

154